# API TESTER - HEALTH INSURANCE CROSS-SELL

In [5]:
import pandas as pd
import requests
import json

In [13]:
homepath = '/home/gutto/Repos/Insurance-Cross-Sell/'

df_test = pd.read_pickle(homepath + 'data/processed/x_val.pkl')
df_test = df_test.sample(50)

In [6]:
# convert dataframe to json
data = json.dumps(df_test.to_dict( orient='records'))

In [7]:
data

'[{"id": 249800, "gender": "Female", "age": 23, "region_code": 49.0, "policy_sales_channel": 152.0, "driving_license": 1, "vehicle_age": "below_1_year", "vehicle_damage": "No", "previously_insured": 0, "annual_premium": 2630.0, "vintage": 245, "response": 0, "premium_per_vintage": 10.73469387755102, "insurance_age": 5, "premium_per_age": 30536.526998744244}, {"id": 39156, "gender": "Male", "age": 38, "region_code": 28.0, "policy_sales_channel": 154.0, "driving_license": 1, "vehicle_age": "between_1_2_year", "vehicle_damage": "Yes", "previously_insured": 0, "annual_premium": 58292.0, "vintage": 81, "response": 0, "premium_per_vintage": 719.6543209876543, "insurance_age": 2, "premium_per_age": 28745.821052631578}, {"id": 14648, "gender": "Male", "age": 24, "region_code": 8.0, "policy_sales_channel": 152.0, "driving_license": 1, "vehicle_age": "below_1_year", "vehicle_damage": "No", "previously_insured": 1, "annual_premium": 27520.0, "vintage": 85, "response": 0, "premium_per_vintage": 32

In [22]:
import os
import pickle
import pandas as pd
from flask import Flask, request, Response
from HealthInsurance import HealthInsurance

path = '/home/gutto/Repos/Insurance-Cross-Sell/src/model/cross_sell.pkl'
#path = '/model/cross_sell.pkl'

# loading model
model = pickle.load(open(path, 'rb'))
print('Loaded!')

print('\n----- INITIALIZING SERVER -----\n')

# API
app = Flask(__name__)

@app.route('/predict', methods = ['POST'])
def healthinsurance_prediction():
    test_json = request.get_json()

    if test_json:
        if isinstance(test_json, dict): # unique example
            test_raw = pd.DataFrame(test_json, index = [0])
        else: # multiple examples
            test_raw = pd.DataFrame(test_json, columns = test_json[0].keys())
        
        # Healthinsurance class
        pipeline = HealthInsurance()

        # data cleaning
        df1 = pipeline.data_cleaning(test_raw)

        # feature_engineering
        df2 = pipeline.feature_engineering(df1)

        # data preparation
        df3 = pipeline.data_preparation(df2)

        # prediction
        df_response = pipeline.get_prediction(model, test_raw, df3)
        
        return df_response

    else:
        return Response('{}', status = 200, mimetype = 'application/json')


if __name__ == '__main__':
    port = os.environ.get('PORT', 5000)
    app.run('0.0.0.0.', port = port, debug = True)

Loaded!

----- INITIALIZING SERVER -----

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://0.0.0.0.:5000/ (Press CTRL+C to quit)
 * Restarting with stat
/home/gutto/.pyenv/versions/3.9.9/envs/pa004/lib/python3.9/site-packages/traitlets/traitlets.py:2202: FutureWarning: Supporting extra quotes around strings is deprecated in traitlets 5.0. You can use 'hmac-sha256' instead of '"hmac-sha256"' if you require traitlets >=5.
  warn(
/home/gutto/.pyenv/versions/3.9.9/envs/pa004/lib/python3.9/site-packages/traitlets/traitlets.py:2157: FutureWarning: Supporting extra quotes around Bytes is deprecated in traitlets 5.0. Use '16bc28e2-c0f9-420a-b8a0-981f490b2236' instead of 'b"16bc28e2-c0f9-420a-b8a0-981f490b2236"'.
  warn(
Traceback (most recent call last):
  File "/home/gutto/.pyenv/versions/3.9.9/envs/pa004/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/gutto/.pyenv/versions/3.9.9/envs/pa004/lib/python3.9/site-packages/traitlets/config/application.py", line 845, in launch_instance
    app.init

SystemExit: 1

/home/gutto/.pyenv/versions/3.9.9/envs/pa004/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [23]:
# API Call
url = 'http://0.0.0.0.:5000/'
#url = 'https://ins-cross-sell.herokuapp.com/'
header = {'Content-type': 'application/json'}

r = requests.post(url, data=data, headers=header)
print(f'Status Code {r.status_code}')

KeyboardInterrupt: 

In [16]:
df1 = pd.DataFrame(r.json(), columns=r.json()[0].keys())
df1.sort_values('score', ascending=False).head()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)